<a href="https://colab.research.google.com/github/werowe/HypatiaAcademy/blob/master/spark/homework_spark_04_03_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing the data
!wget https://raw.githubusercontent.com/illiazhupanov/pandas/refs/heads/main/steam_games.csv

--2025-03-06 17:12:53--  https://raw.githubusercontent.com/illiazhupanov/pandas/refs/heads/main/steam_games.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20395976 (19M) [text/plain]
Saving to: ‘steam_games.csv’

steam_games.csv     100%[===================>]  19.45M   125MB/s    in 0.2s    

2025-03-06 17:12:54 (125 MB/s) - ‘steam_games.csv’ saved [20395976/20395976]



In [ ]:
!ls

sample_data  steam_games.csv


In [ ]:
# added this to remove the file
#!rm steam_games.csv

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('spark').getOrCreate()

In [ ]:
df = spark.read.csv('steam_games.csv', header = True, inferSchema = True, sep = ',', encoding="UTF-8")
df.printSchema()

root
 |-- steam_appid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- developers: string (nullable = true)
 |-- publishers: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- required_age: string (nullable = true)
 |-- n_achievements: string (nullable = true)
 |-- platforms: string (nullable = true)
 |-- is_released: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- additional_content: string (nullable = true)
 |-- total_reviews: string (nullable = true)
 |-- total_positive: string (nullable = true)
 |-- total_negative: string (nullable = true)
 |-- review_score: string (nullable = true)
 |-- review_score_desc: string (nullable = true)
 |-- positive_percentual: string (nullable = true)
 |-- metacritic: string (nullable = true)
 |-- is_free: string (nullable = true)
 |-- price_initial (USD): string (nullable = true)



In [ ]:
from pyspark.sql.functions import regexp_replace, col
# renaming one of the columns

# get rid of space

df = df.withColumnRenamed('price_initial (USD)', 'price_initial')
df.printSchema()

root
 |-- steam_appid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- developers: string (nullable = true)
 |-- publishers: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- required_age: string (nullable = true)
 |-- n_achievements: string (nullable = true)
 |-- platforms: string (nullable = true)
 |-- is_released: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- additional_content: string (nullable = true)
 |-- total_reviews: string (nullable = true)
 |-- total_positive: string (nullable = true)
 |-- total_negative: string (nullable = true)
 |-- review_score: string (nullable = true)
 |-- review_score_desc: string (nullable = true)
 |-- positive_percentual: string (nullable = true)
 |-- metacritic: string (nullable = true)
 |-- is_free: string (nullable = true)
 |-- price_initial: string (nullable = true)



it seems that we need to call

'df.createOrReplaceTempView("games")'

every time when we update a dataframe

cuz if we don't, it will just show the last updated version


In [ ]:
df.createOrReplaceTempView("games")

com = '''
select name, release_date from games
where release_date like '%00:00:00%'
''' # we can use the like operator to do a sort of filtering rows by the fact whether they contain a particular string in them (kinda like regex)
# in this case we are telling it to find only the rows that contain 00:00:00 in the string value in the release_date column
spark.sql(com).show()

+--------------------+-------------------+
|                name|       release_date|
+--------------------+-------------------+
|      Sexy Blackjack|2024-01-25 00:00:00|
|They Came From Venus|2024-07-22 00:00:00|
|         Bubble Wrap|2024-09-19 00:00:00|
|           It Mimics|2023-12-25 00:00:00|
|        Lost Mansion|2023-12-22 00:00:00|
|            热血大唐|2024-04-24 00:00:00|
|     Boundless Skies|2023-12-20 00:00:00|
|           Ace Triad|2024-01-22 00:00:00|
| Apathy In Our Stars|2024-01-15 00:00:00|
| SEARCH ALL - SNAILS|2023-12-18 00:00:00|
|Christmas Fables:...|2023-12-27 00:00:00|
|        The Skirmish|2024-01-05 00:00:00|
|Mortui: Outbreak ...|2024-02-19 00:00:00|
|Escape from Black...|2024-01-01 00:00:00|
|               Maze!|2023-12-27 00:00:00|
|        Tipsy Rescue|2024-02-24 00:00:00|
|         Void Reaper|2024-09-01 00:00:00|
| The Last Human: GO!|2024-01-08 00:00:00|
|  Roll Over Princess|2024-02-13 00:00:00|
| Tobla - Divine Path|2024-03-29 00:00:00|
+--------------

In [ ]:
com = '''
select name, release_date from games

''' # we can use the like operator to do a sort of filtering rows by the fact whether they contain a particular string in them (kinda like regex)
# in this case we are telling it to find only the rows that contain 00:00:00 in the string value in the release_date column
spark.sql(com).show()

+--------------------+------------+
|                name|release_date|
+--------------------+------------+
|    勇者の伝説の勇者|  2024-01-04|
|       Light No Fire|Not Released|
|Lorhaven: Cursed War|  2024-01-26|
|        PUIQ: Demons|  2024-02-17|
|      Project XSTING|  2024-01-05|
|The Milgram Exper...|Not Released|
| 5 Bucks for the Bus|Not Released|
|       Manor Madness|  2024-01-15|
|           Fly Hands|  2024-03-26|
|      Zero Caliber 2|Not Released|
|          Star Birds|Not Released|
|CLIMB UP! Battle ...|Not Released|
|            Tafl PTK|  2024-02-29|
|Striker Manager 3...|  2024-05-23|
|Fantastic Haven: ...|  2024-06-08|
|Super Spelling Ducks|Not Released|
|            Solstale|  2024-04-06|
|   Asylum Nightmares|  2024-07-01|
|           Unveiling|  2024-01-08|
|      Hotdog Kitchen|  2024-04-04|
+--------------------+------------+
only showing top 20 rows



In [ ]:
df = df.withColumn('release_date', regexp_replace(col('release_date'), '\s\d\d:\d\d:\d\d', '')) # removing the 00:00:00 -like values from the release_date
df = df.withColumn('release_date', regexp_replace(col('release_date'), 'Not Released', ''))
df.createOrReplaceTempView("games") # creating a temp view of the updated df

# this now doesn't return any values, that means that we got rid of the times
com = '''
select name, release_date from games
where release_date like '%00:00:00%' or release_date like ''
'''
spark.sql(com).show()

+--------------------+------------+
|                name|release_date|
+--------------------+------------+
|       Light No Fire|            |
|The Milgram Exper...|            |
| 5 Bucks for the Bus|            |
|      Zero Caliber 2|            |
|          Star Birds|            |
|CLIMB UP! Battle ...|            |
|Super Spelling Ducks|            |
|        Neon Goddess|            |
|          Tearfallen|            |
|              Return|            |
|    Spirits of Baciu|            |
|          The Rising|            |
|             The Mom|            |
|      Lottery Winner|            |
|Legends of Awen: ...|            |
|   Crusader's Legacy|            |
|                Into|            |
| RUVN Contest Reboot|            |
|Xstall One - eVTO...|            |
|         Green Again|            |
+--------------------+------------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import to_date
# I used the to_date func instead of cast because it converts any values that are formatted inconsistently (Not Released in this case) to null
df = df.withColumn('release_date', to_date(col('release_date'), 'yyyy-MM-dd'))

df.printSchema()

root
 |-- steam_appid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- developers: string (nullable = true)
 |-- publishers: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- required_age: string (nullable = true)
 |-- n_achievements: string (nullable = true)
 |-- platforms: string (nullable = true)
 |-- is_released: string (nullable = true)
 |-- release_date: date (nullable = true)
 |-- additional_content: string (nullable = true)
 |-- total_reviews: string (nullable = true)
 |-- total_positive: string (nullable = true)
 |-- total_negative: string (nullable = true)
 |-- review_score: string (nullable = true)
 |-- review_score_desc: string (nullable = true)
 |-- positive_percentual: string (nullable = true)
 |-- metacritic: string (nullable = true)
 |-- is_free: string (nullable = true)
 |-- price_initial: string (nullable = true)



In [ ]:
df = df.withColumn('is_released', col('is_released').cast('boolean'))
df.printSchema()
df.createOrReplaceTempView('games')
com = '''
select is_released, count(is_released) as cnt from games
where price_initial > 10.0
group by is_released
'''
spark.sql(com).show()

root
 |-- steam_appid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- developers: string (nullable = true)
 |-- publishers: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- required_age: string (nullable = true)
 |-- n_achievements: string (nullable = true)
 |-- platforms: string (nullable = true)
 |-- is_released: boolean (nullable = true)
 |-- release_date: date (nullable = true)
 |-- additional_content: string (nullable = true)
 |-- total_reviews: string (nullable = true)
 |-- total_positive: string (nullable = true)
 |-- total_negative: string (nullable = true)
 |-- review_score: string (nullable = true)
 |-- review_score_desc: string (nullable = true)
 |-- positive_percentual: string (nullable = true)
 |-- metacritic: string (nullable = true)
 |-- is_free: string (nullable = true)
 |-- price_initial: string (nullable = true)

+-----------+-----+
|is_released|  cnt|
+-----------+-----+
|       true|121

In [ ]:
df.createOrReplaceTempView("games")
# now we can see that all the correctly formatted dates were left, but the Not Released strings turned into null
com = '''
select release_date from games
'''
spark.sql(com).show()

In [ ]:
df.createOrReplaceTempView("games")
# we can also call only the null values with sql
com = '''
select name, release_date from games
where release_date IS NULL
'''
spark.sql(com).show()

In [ ]:
# I also converted the price_initial to be float instead of string
df = df.withColumn('price_initial', col('price_initial').cast('float'))
df.printSchema()

root
 |-- steam_appid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- developers: string (nullable = true)
 |-- publishers: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- required_age: string (nullable = true)
 |-- n_achievements: string (nullable = true)
 |-- platforms: string (nullable = true)
 |-- is_released: boolean (nullable = true)
 |-- release_date: date (nullable = true)
 |-- additional_content: string (nullable = true)
 |-- total_reviews: string (nullable = true)
 |-- total_positive: string (nullable = true)
 |-- total_negative: string (nullable = true)
 |-- review_score: string (nullable = true)
 |-- review_score_desc: string (nullable = true)
 |-- positive_percentual: string (nullable = true)
 |-- metacritic: string (nullable = true)
 |-- is_free: string (nullable = true)
 |-- price_initial: float (nullable = true)



In [ ]:
df.createOrReplaceTempView("games")

# we can now filter by price in sql statements
com = '''
select name, price_initial from games
where price_initial < 10 and price_initial > 8
order by price_initial asc
'''
spark.sql(com).show()

In [ ]:
df.createOrReplaceTempView("games")
# here I displayed the sum of prices of all games per platforms they are available on
# since platforms is still a str, we have to use 'like' in the sql statements
com = '''
select sum(price_initial) as total_price, sum(case when is_released is TRUE then 1 else 0 end) as released, sum(case when is_released is FALSE then 1 else 0 end) as not_released
from games
where platforms like '%windows%' or platforms like '%linux%' or platforms like '%mac%'
group by platforms
'''
spark.sql(com).show()

+------------------+--------+------------+
|       total_price|released|not_released|
+------------------+--------+------------+
| 837208.5765452981|   44026|       10830|
| 34.97999954223633|       3|           3|
| 42909.98935389519|    5538|         994|
|208.91000199317932|      13|           8|
|13505.539759159088|    2065|         546|
| 74798.88980722427|    6352|        1004|
| 4.989999771118164|       1|           0|
+------------------+--------+------------+



In [ ]:
com = '''
select sum(case when is_released is FALSE then 1 end) as count from games
where is_released is FALSE
'''
spark.sql(com).show()

+-----+
|count|
+-----+
|13388|
+-----+

